In [ ]:
import ipywidgets as widgets
from IPython.display import display

# File picker widget to browse for a MIDI file
uploader = widgets.FileUpload(accept='.mid', multiple=False)
display(uploader)

In [ ]:
# Cell 2: Retrieve Uploader Value and Convert to CSV
import py_midicsv
import tempfile

# Function to handle the uploaded MIDI file and convert to CSV
def handle_midi_upload():
    if uploader.value:
        uploaded_file = uploader.value[0]
        
        # Write the content to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mid") as temp_midi_file:
            temp_midi_file.write(uploaded_file['content'])
            temp_midi_file_path = temp_midi_file.name

        # Convert the MIDI file to CSV format
        csv_string = py_midicsv.midi_to_csv(temp_midi_file_path)

        # Join the CSV data to create a single string
        csv_data = "\n".join(csv_string)

        # Print or save the CSV data
        print(f"MIDI file '{uploaded_file['name']}' has been converted to CSV:")
        #print(csv_data)

        return csv_data
    else:
        print("No file uploaded. Please upload a valid MIDI file.")
        return None


In [ ]:
# Cell 3: Clean CSV and Extract Metadata to Variables
# Function to clean the CSV and extract metadata
def clean_csv_and_extract_metadata(csv_data):
    if csv_data is None:
        print("CSV data is not available. Please upload a MIDI file and convert it to CSV first.")
        return None, None

    rows = csv_data.splitlines()
    metadata = []
    midi_data = []

    for row in rows:
        elements = row.split(", ")
        if len(elements) < 3 or elements[2] not in ["Note_on_c", "Note_off_c"]:
            metadata.append(row)
        else:
            midi_data.append(row)

    # Remove empty rows from metadata and midi data
    metadata = [row for row in metadata if row.strip()]
    midi_data = [row for row in midi_data if row.strip()]

    metadata_csv = "\n".join(metadata)
    midi_csv = "\n".join(midi_data)

    print("Metadata and cleaned MIDI CSV data have been extracted to variables.")

    return metadata_csv, midi_csv


In [ ]:
# Call the function to convert MIDI to CSV and then clean the data
csv_data = handle_midi_upload()
metadata_csv, midi_csv = clean_csv_and_extract_metadata(csv_data)

print(metadata_csv)
print(midi_csv)

In [ ]:
# Cell 4: Parse Useful Data from Metadata
# Function to parse useful data from metadata CSV
def parse_metadata(metadata_csv):
    rows = metadata_csv.splitlines()

    # Extract division from the header row (first row)
    header_elements = rows[0].split(", ")
    division = int(header_elements[5])

    # Extract total pulses from the second to last row
    second_last_row_elements = rows[-2].split(", ")
    total_pulses = int(second_last_row_elements[1])

    # Iterate through rows to find Time_signature and extract elements 3 and 4
    time_signature_numerator = None #beats per measure
    time_signature_beatID = None    #2^beatID = demonimator (i.e. quarterNoteID = 2, eighthNoteId = 3)

    for row in rows:
        elements = row.split(", ")
        if len(elements) >= 3 and elements[2] == "Time_signature":
            time_signature_numerator = int(elements[3])
            time_signature_beatID = int(elements[4])
            break

    #print(f"Division: {division}")
    #print(f"Total Pulses: {total_pulses}")

    return division, total_pulses, time_signature_numerator, time_signature_beatID


In [ ]:
# Cell 5: Calculate Musical Grid Structure
# This cell calculates the total number of measures, beats per measure, and rows needed for visualization.
import math

pulses_per_quarterNote, total_pulses, time_sig_numerator, time_sig_beatID = parse_metadata(metadata_csv)

#LATER:
#time_sig_demonimator = 2 ** time_sig_beatID ...
#NOW:
# Assuming 4/4 time signature
beats_per_measure = 4

total_quarterNotes = math.ceil(total_pulses / pulses_per_quarterNote)
total_measures = total_quarterNotes / beats_per_measure


#Calculate BOLD GRIDLINES
measures_per_row = 16
row_count = math.ceil(total_measures / measures_per_row)

print(f"Beats per Measure: {beats_per_measure}")
print(f"Total Quarter Notes: {total_quarterNotes}")
print(f"Total Measures: {total_measures}")
print(f"Measures per Row: {measures_per_row}")
print(f"Row Count: {row_count}")





In [ ]:
# Cell 6: Convert MIDI Absolute Clock Time to Pixel Coordinates
# Function to convert MIDI absolute clock time to pixel values on the grid

def convert_midi_time_to_pixel_coordinates(midi_csv, canvas_width, canvas_height, row_count=row_count, measures_per_row=measures_per_row):
    rows = midi_csv.splitlines()
    pixel_coordinates = []

    # Define the dimensions of each grid section
    measure_width = canvas_width / measures_per_row
    row_height = canvas_height / row_count

    for row in rows:
        elements = row.split(", ")
        
        # Process only "Note_on_c" events
        if len(elements) >= 4 and elements[2] == "Note_on_c" and int(elements[5]) != 0:
            absolute_time = int(elements[1])
            
            # Determine row and column based on absolute time
            total_quarter_notes = absolute_time / pulses_per_quarterNote
            measure_number = total_quarter_notes / beats_per_measure

            row_number = int(measure_number // measures_per_row)
            column_within_row = measure_number % measures_per_row

            # Convert to pixel values
            x_pixel = column_within_row * measure_width
            y_pixel = (row_number + 0.5) * row_height  # Place event in the middle of its respective row

            pixel_coordinates.append((x_pixel, y_pixel))

    return pixel_coordinates


In [ ]:
# Call the Conversion Function and Store Pixel Coordinates
# Call the function to convert MIDI absolute clock time to pixel coordinates
canvas_width = 800
canvas_height = 400

pixel_coordinates = convert_midi_time_to_pixel_coordinates(
    midi_csv=midi_csv,
    canvas_width = canvas_width,
    canvas_height = canvas_height,
    row_count=row_count,
    measures_per_row=measures_per_row
)


In [ ]:
# Visualization with p5.js - Draw Grid and Notes
from IPython.display import HTML

# Convert pixel_coordinates list to a JavaScript array representation
js_pixel_coordinates = str(pixel_coordinates).replace("(", "[").replace(")", "]")

html_code = f"""
<div id="p5_container"></div>
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/1.4.0/p5.min.js"></script>
<script>
    new p5((sketch) => {{
        let measuresPerRow = {measures_per_row};
        let rowCount = {row_count};
        let beatsPerMeasure = {beats_per_measure};
        let canvasWidth = {canvas_width};
        let canvasHeight = {canvas_height};

        // Pixel coordinates for notes (from Python)
        let pixelCoordinates = {js_pixel_coordinates};

        sketch.setup = function() {{
            let canvas = sketch.createCanvas(canvasWidth, canvasHeight);
            canvas.parent('p5_container');
            sketch.background(255);
        }};

        sketch.draw = function() {{
            sketch.clear();
            sketch.background(255);

            // Draw gridlines for measures and beats
            let rowHeight = sketch.height / rowCount;
            let measureWidth = sketch.width / measuresPerRow;
            let beatWidth = measureWidth / beatsPerMeasure;

            // Draw rows
            sketch.stroke(0);
            sketch.strokeWeight(2);
            for (let i = 0; i <= rowCount; i++) {{
                sketch.line(0, i * rowHeight, sketch.width, i * rowHeight);
            }}

            // Draw measures and beats
            for (let j = 0; j <= measuresPerRow; j++) {{
                let x = j * measureWidth;

                // Bold lines for measures
                sketch.stroke(0);
                sketch.strokeWeight(2);
                sketch.line(x, 0, x, sketch.height);

                // Lighter lines for beats within each measure
                if (j < measuresPerRow) {{
                    for (let k = 1; k < beatsPerMeasure; k++) {{
                        let beatX = x + k * beatWidth;
                        sketch.stroke(200);
                        sketch.strokeWeight(1);
                        sketch.line(beatX, 0, beatX, sketch.height);
                    }}
                }}
            }}

            // Draw notes from pixel coordinates
            sketch.fill(100, 100, 255, 255);
            sketch.noStroke();
            pixelCoordinates.forEach(([x, y]) => {{
                sketch.ellipse(x, y, 10, 10); // Draw a circle for each note
            }});
        }};
    }});
</script>
"""

display(HTML(html_code))
